In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd

In [2]:
from upath import UPath

dirpath_raw = "s3://prescient-lobster/ume/datasets/atomica/raw"

files_raw = list(UPath(dirpath_raw).glob("*.csv"))

ds = []

for file in files_raw:
    print(file)
    _df = pd.read_csv(file, sep="\t") if "small_molecule-small_molecule" not in file.stem else pd.read_csv(file, sep=",")

    _df.insert(0, "interaction_type", file.stem.replace(".csv", ""))
    _df.drop(columns=["pdb_id"], inplace=True, errors="ignore")

    print(_df["interaction_type"].value_counts())

    ds.append(_df)

df_raw = pd.concat(ds, ignore_index=True)
df_raw["interaction_type"].value_counts()

s3://prescient-lobster/ume/datasets/atomica/raw/protein-dna.csv
interaction_type
protein-dna    2750
Name: count, dtype: int64
s3://prescient-lobster/ume/datasets/atomica/raw/protein-ion.csv
interaction_type
protein-ion    74514
Name: count, dtype: int64
s3://prescient-lobster/ume/datasets/atomica/raw/protein-peptide.csv
interaction_type
protein-peptide    8475
Name: count, dtype: int64
s3://prescient-lobster/ume/datasets/atomica/raw/protein-protein.csv
interaction_type
protein-protein    124541
Name: count, dtype: int64
s3://prescient-lobster/ume/datasets/atomica/raw/protein-rna.csv
interaction_type
protein-rna    3511
Name: count, dtype: int64
s3://prescient-lobster/ume/datasets/atomica/raw/protein-small_molecule.csv
interaction_type
protein-small_molecule    119017
Name: count, dtype: int64
s3://prescient-lobster/ume/datasets/atomica/raw/rna-small_molecule.csv
interaction_type
rna-small_molecule    5185
Name: count, dtype: int64
s3://prescient-lobster/ume/datasets/atomica/raw/small_

interaction_type
small_molecule-small_molecule    1767710
protein-protein                   124541
protein-small_molecule            119017
protein-ion                        74514
protein-peptide                     8475
rna-small_molecule                  5185
protein-rna                         3511
protein-dna                         2750
Name: count, dtype: int64

In [5]:
df_raw[df_raw["interaction_type"] == "protein-small_molecule"]

,interaction_type,split,id
213791,protein-small_molecule,train,4h4e_1.pdb_4h4e_1_10G_D.pdb
213792,protein-small_molecule,train,4h4e_1.pdb_4h4e_1_SF4_C.pdb
213793,protein-small_molecule,train,6llw_2.pdb_6llw_2_UDP_D.pdb
213794,protein-small_molecule,train,8sdb_4.pdb_8sdb_4_GLC_T.pdb
213795,protein-small_molecule,train,3tz9_1.pdb_3tz9_1_AQU_C.pdb
...,...,...,...
332803,protein-small_molecule,test,6pw5_1.pdb_6pw5_1_PIO_N.pdb
332804,protein-small_molecule,test,6pw5_1.pdb_6pw5_1_PIK_M.pdb
332805,protein-small_molecule,test,6pw5_1.pdb_6pw5_1_PIO_E.pdb
332806,protein-small_molecule,test,6pw5_1.pdb_6pw5_1_PIO_H.pdb


In [7]:
from lobster.tmp import process_atomica

process_atomica._fetch_pdb_data("6pw5")

[{'molecule_type': 'polypeptide(L)',
  'entity_id': 1,
  'sample_preparation': 'Genetically manipulated',
  'length': 901,
  'number_of_copies': 4,
  'in_chains': ['A', 'B', 'C', 'D'],
  'in_struct_asyms': ['A', 'B', 'C', 'D'],
  'mutation_flag': None,
  'weight': 99538.742,
  'ca_p_only': False,
  'synonym': 'TRP-like ion channel',
  'molecule_name': ['Ion transport domain-containing protein'],
  'gene_name': ['CHLRE_10g452950v5'],
  'source': [{'organism_scientific_name': 'Chlamydomonas reinhardtii',
    'expression_host_scientific_name': 'Homo sapiens',
    'tax_id': 3055,
    'expression_host_tax_id': 9606,
    'mappings': [{'start': {'residue_number': 1},
      'end': {'residue_number': 901}}]}],
  'sequence': 'MKVAPAPASGQPGGGSKKPSDYGCQLHYKHARVVEPESTTDDGMKRLKDVGDKGTLITAAELGLVDKYRDLKRAGQDILTCDWPYHYSSILYACYGNQYKILQMVEREFVGSTQELTAMHTTRCWVGKNSAMVAAYQGHLETMLYIIDLDMQGKFTEDLFKQRDVMGKNAMMWAASQGHTDTIEVLLVRSLYRLLPEDCADPLVLKTRWKLVSLLADLASHCRDYDPGCSRSFFQEVLASIKYDPVEGARQEEAAAAGGGGSAREGAALHEPTW

### Processed

In [ ]:
from upath import UPath

dirpath = "s3://prescient-lobster/ume/datasets/atomica/pre-processed/"

files = list(UPath(dirpath).glob("*.parquet"))

dfs = []

for file in files:
    _df = pd.read_parquet(file)
    _df["interaction_type"] = file.name.split(".")[0]
    dfs.append(_df)

df_og = pd.concat(dfs, ignore_index=True)

In [ ]:
# df = df_og[["sequence1", "modality1", "sequence2", "modality2", "interaction_type"]]

df = df_og.replace({
    "polydeoxyribonucleotide": "nucleotide",
    "polydeoxyribonucleotide/polyribonucleotide hybrid ": "nucleotide",
    "polyribonucleotide": "nucleotide",
})

In [ ]:
from sklearn.model_selection import train_test_split

df_train_val, df_test = train_test_split(df, test_size=0.05, random_state=0, stratify=df["interaction_type"])
df_train, df_val = train_test_split(df_train_val, test_size=0.05, random_state=0, stratify=df_train_val["interaction_type"])

In [ ]:
df_train["split"] = "train"
df_val["split"] = "val"
df_test["split"] = "test"

df = pd.concat([df_train, df_val, df_test], ignore_index=True)

In [ ]:
pd.notnull(df)

In [ ]:
df = df.where(pd.notnull(df), None)
df = df.dropna(axis=0, how="all")
df = df.dropna(axis=1, how="all")

df

# column order:
df = df[["interaction_type", "sequence1", "modality1", "sequence2", "modality2", "sequence3", "modality3", "sequence4", "modality4", "sequence5", "modality5", "split"]]
df

In [ ]:
df = df.sort_values(by=["interaction_type", "split"]).reset_index(drop=True)
df

In [ ]:
df.groupby("interaction_type").head(1).reset_index(drop=True)

In [ ]:
df[["interaction_type"]].value_counts().sort_index()

In [ ]:
df.to_parquet(
    "s3://prescient-lobster/ume/datasets/atomica/processed",
    index=False,
    engine="pyarrow",
    partition_cols=["split", "interaction_type"],
)

In [20]:
df = pd.read_parquet(
    "s3://prescient-lobster/ume/datasets/atomica/processed",
)

In [30]:
df.groupby("interaction_type").head(1).reset_index(drop=True)

/var/folders/zv/m9rwl6cx1qd73n1yn3vdrm_c0000gp/T/ipykernel_4855/2657167201.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("interaction_type").head(1).reset_index(drop=True)


,sequence1,modality1,sequence2,modality2,sequence3,modality3,sequence4,modality4,sequence5,modality5,split,interaction_type
0,MAGVKNSIIWFRKGLRLHDNPALLEACKDAKHVYPVFVLDPHFLQQ...,amino_acid,CAGCGGTTGCCGTG,nucleotide,CACGGCAACCGCTG,nucleotide,None,None,None,None,test,protein-dna
1,GPLGSPEFGRPGWVIGVNPDIGGAIAVLSPDGSSQVFDNPFVHIVV...,amino_acid,GPLGSPEFGRPGWVIGVNPDIGGAIAVLSPDGSSQVFDNPFVHIVV...,amino_acid,None,None,None,None,None,None,test,protein-protein
2,MHHHHHHENLYFQGSGMAGSVGLALCGQTLVVRGGSRFLATSIASS...,amino_acid,MAAETRNVAGAEAPPPQKRYYRQRAHSNPMADHTLRYPVKPEEMDW...,amino_acid,GCCCGGAUAGCUCAGUCGGUAGAGCAUCAGACUUUUAAUCUGAGGG...,nucleotide,None,None,None,None,test,protein-rna
3,MTSNNLPTVLESIVEGRRGHLEEIRARIAHVDVDALPKSTRSLFDS...,amino_acid,O=C(O)c1ccccc1NC[C@@H](O)[C@H](O)[C@H](O)COP(=...,smiles,None,None,None,None,None,None,test,protein-small_molecule
4,c1ccccc1,smiles,Cc1ccc([NH+]=P(C)(C)c2ccccc2)c(c1)S(=O)(=O)[O-],smiles,None,None,None,None,None,None,test,small_molecule-small_molecule


In [24]:
df[df["sequence3"].isna()]

,sequence1,modality1,sequence2,modality2,sequence3,modality3,sequence4,modality4,sequence5,modality5,split,interaction_type
5,MKHSFSRFFGLGEKEQEPEIAEHDTNKEEILEIPVNAIVPNRFQPR...,amino_acid,TATTTCCCGGGAAATA,nucleotide,None,None,None,None,None,None,test,protein-dna
6,MTSMASLFSFTSPAVKRLLGWKQGDEEEKWAEKAVDALVKKLKKKK...,amino_acid,GTATGGCGCCATAC,nucleotide,None,None,None,None,None,None,test,protein-dna
7,MVEIILSHLIFDQAYFSKVWPYMDSEYFESGPAKNTFKLIKSHVNE...,amino_acid,TTTTTTTTTT,nucleotide,None,None,None,None,None,None,test,protein-dna
11,MRTELLSKLYDDFGIDQLPHTQHGVTSDRLGKLYEKYILDIFKDIE...,amino_acid,CCCCCGGGGG,nucleotide,None,None,None,None,None,None,test,protein-dna
12,GTNRRQNRKCGACAACLRRMDCGRCDFCCDKPKFGGSNQKRQKCRW...,amino_acid,GCCAACGTTGGC,nucleotide,None,None,None,None,None,None,test,protein-dna
...,...,...,...,...,...,...,...,...,...,...,...,...
343339,[nH+]1ccccc1c1nc(nc(n1)c1ccccn1)c1[nH+]cccc1,smiles,F[P-](F)(F)(F)(F)F,smiles,None,None,None,None,None,None,val,small_molecule-small_molecule
343340,OC1(CCCC(O)(C1)C(F)(F)C(F)C(F)(F)F)C(F)(F)C(F)...,smiles,OC1(CCCC(O)(C1)C(F)(F)[C](F)C(F)(F)F)C(F)(F)[C...,smiles,None,None,None,None,None,None,val,small_molecule-small_molecule
343341,O=C([O-])c1ccc(cc1)C(=O)C1C2CCC1C=C2,smiles,[NH3+]C(C)C1CCCCC1,smiles,None,None,None,None,None,None,val,small_molecule-small_molecule
343342,OC(=O)c1ccccc1O,smiles,OC(=O)C1=CN(C2CC2)c2cc(N3CC[NH2+]CC3)c(F)cc2C1=O,smiles,None,None,None,None,None,None,val,small_molecule-small_molecule


In [26]:
max_modalities = 2

[
    (f"sequence{i}", f"modality{i}") for i in range(1, max_modalities + 1)
]

[('sequence1', 'modality1'), ('sequence2', 'modality2')]

In [28]:
columns = [item for i in range(1, max_modalities + 1) for item in (f"sequence{i}", f"modality{i}")]
columns

['sequence1', 'modality1', 'sequence2', 'modality2']